In [ ]:
import os
import matplotlib.pyplot as plt

!pip install nnunetv2 --quiet
!pip install pydicom --quiet
!pip install boxsdk --quiet
!pip install pynrrd --quiet

import nnunetv2
import numpy as np
import pydicom
import zipfile
import re
import gzip
import matplotlib.pyplot as plt
import pandas as pd
import nrrd
import boxsdk
from boxsdk import Client, OAuth2


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import json

In [ ]:
runtime_dir = '/content/runtime_files/'
files_dir = '/content/runtime_files/data_dir'
data_dir = '/content/runtime_files/data_dir/nnUNet_raw'


if not os.path.exists(runtime_dir):
    os.mkdir(runtime_dir)
if not os.path.exists(files_dir):
    os.mkdir(files_dir)
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [ ]:
patient_id_555 = []
patient_id_556 = []
test_scores_dice_555 = []
test_scores_dice_556 = []

In [ ]:
dataset = "Dataset555_OPCONTRAST"

imagesTs_dir = os.path.join('/content/runtime_files/data_dir/nnUNet_raw',dataset,'imagesTs')
labelsTs_gt_dir = os.path.join('/content/runtime_files/data_dir/nnUNet_raw',dataset,'labelsTs')

imagesTs_files = os.listdir(imagesTs_dir)
labelsTs_gt_files = os.listdir(labelsTs_gt_dir)


# results_dir =  ''
labelsTs_pred_dir = os.path.join(results_dir, dataset, 'test_predictions_3d_fullres')
labelsTs_pred_files = [filename for filename in os.listdir(labelsTs_pred_dir) if '.nii.gz' in filename]



# print(imagesTs_files)
# print(labelsTs_gt_files)
#print(labelsTs_pred_files)


with open(os.path.join(labelsTs_pred_dir, 'summary.json'), 'r') as f:
    test_results_summary_data = json.load(f)
    #print(data['foreground_mean'])
    print("OVERALL DICE SCORE: "+str(round(test_results_summary_data['foreground_mean']['Dice'],2)))
    print("OVERALL IOU SCORE: "+str(round(test_results_summary_data['foreground_mean']['IoU'],2)))

for imagesTs_file in imagesTs_files:
    print("\n\n")
    # CASE SPECIFIC SUMMARY & VISUALIZATION
    #print(imagesTs_file)
    label_name = imagesTs_file.replace('_0000','')
    #print(label_name)
    scan_id = "_"+str(label_name.split('_')[1].replace('.nii.gz',''))
    #print(scan_id)

    patient_id = "IRB-0749:"+str(patient_ids_lookup[patient_ids_lookup['Patient_ID'] == scan_id]['patients'].iloc[0])
    print(patient_id)
    for i in range(len(test_results_summary_data['metric_per_case'])):
        if scan_id == "_"+test_results_summary_data['metric_per_case'][i]['prediction_file'].split('_')[-1].replace(".nii.gz",""):
            print("DICE SCORE: "+str(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['Dice'],2)))
            print("IOU SCORE: "+str(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['IoU'],2)))
            patient_id_555.append(patient_id)
            test_scores_dice_555.append(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['Dice'],2))

    image = np.rot90(nib.load(os.path.join(imagesTs_dir, imagesTs_file)).get_fdata(), k=-1, axes=(0, 1))
    label_gt = np.rot90(nib.load(os.path.join(labelsTs_gt_dir, label_name)).get_fdata(), k=-1, axes=(0, 1))
    label_pred = np.rot90(nib.load(os.path.join(labelsTs_pred_dir, label_name)).get_fdata(), k=-1, axes=(0, 1))

    # print(image.shape)
    # print(label_gt.shape)
    # print(label_pred.shape)

    # find max mask slice and plot 3 slices above and below max mask slice

    max_mask_slice = 0
    max_mask_value = 0

    for i in range(image.shape[2]):
        if np.count_nonzero(label_gt[:,:,i]) > max_mask_value:
            max_mask_slice = i
            max_mask_value = np.count_nonzero(label_gt[:,:,i])

    fig, axes = plt.subplots(2, 5, figsize=(20, 8)) # Increased figure height

    for i, idx in enumerate(range(max_mask_slice - 2, max_mask_slice + 3)):
        if 0 <= idx < image.shape[2]:
            # Row 0: Image and label_gt
            axes[0, i].imshow(image[:, :, idx], cmap='gray')
            axes[0, i].imshow(label_gt[:, :, idx], alpha=0.3) # Reduced alpha for better visibility
            axes[0, i].set_title(f"Ground Truth segmentation Slice {idx}")
            axes[0, i].axis('off')

            # Row 1: Image and label_pred
            axes[1, i].imshow(image[:, :, idx], cmap='gray')
            axes[1, i].imshow(label_pred[:, :, idx], alpha=0.3) # Reduced alpha
            axes[1, i].set_title(f"Predicted Segmentation Slice {idx}")
            axes[1, i].axis('off')

        else:
            axes[0, i].axis('off')
            axes[1, i].axis('off')

    plt.tight_layout() # Adjust subplot params for a tight layout
    plt.show()




In [ ]:
dataset = "Dataset556_OPNONCONTRAST"

imagesTs_dir = os.path.join('/content/runtime_files/data_dir/nnUNet_raw',dataset,'imagesTs')
labelsTs_gt_dir = os.path.join('/content/runtime_files/data_dir/nnUNet_raw',dataset,'labelsTs')

imagesTs_files = os.listdir(imagesTs_dir)
labelsTs_gt_files = os.listdir(labelsTs_gt_dir)


# results_dir =  '/'
labelsTs_pred_dir = os.path.join(results_dir, dataset, 'test_predictions_3d_fullres')
labelsTs_pred_files = [filename for filename in os.listdir(labelsTs_pred_dir) if '.nii.gz' in filename]



# print(imagesTs_files)
# print(labelsTs_gt_files)
#print(labelsTs_pred_files)


with open(os.path.join(labelsTs_pred_dir, 'summary.json'), 'r') as f:
    test_results_summary_data = json.load(f)
    #print(data['foreground_mean'])
    print("OVERALL DICE SCORE: "+str(round(test_results_summary_data['foreground_mean']['Dice'],2)))
    print("OVERALL IOU SCORE: "+str(round(test_results_summary_data['foreground_mean']['IoU'],2)))

for imagesTs_file in imagesTs_files:
    print("\n\n")
    # CASE SPECIFIC SUMMARY & VISUALIZATION
    #print(imagesTs_file)
    label_name = imagesTs_file.replace('_0000','')
    #print(label_name)
    scan_id = "_"+str(label_name.split('_')[1].replace('.nii.gz',''))
    #print(scan_id)

    patient_id = "IRB-0749:"+str(patient_ids_lookup[patient_ids_lookup['Patient_ID'] == scan_id]['patients'].iloc[0])
    print(patient_id)
    for i in range(len(test_results_summary_data['metric_per_case'])):
        if scan_id == "_"+test_results_summary_data['metric_per_case'][i]['prediction_file'].split('_')[-1].replace(".nii.gz",""):
            print("DICE SCORE: "+str(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['Dice'],2)))
            print("IOU SCORE: "+str(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['IoU'],2)))
            patient_id_556.append(patient_id)
            test_scores_dice_556.append(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['Dice'],2))

    image = np.rot90(nib.load(os.path.join(imagesTs_dir, imagesTs_file)).get_fdata(), k=-1, axes=(0, 1))
    label_gt = np.rot90(nib.load(os.path.join(labelsTs_gt_dir, label_name)).get_fdata(), k=-1, axes=(0, 1))
    label_pred = np.rot90(nib.load(os.path.join(labelsTs_pred_dir, label_name)).get_fdata(), k=-1, axes=(0, 1))

    # print(image.shape)
    # print(label_gt.shape)
    # print(label_pred.shape)

    # find max mask slice and plot 3 slices above and below max mask slice

    max_mask_slice = 0
    max_mask_value = 0

    for i in range(image.shape[2]):
        if np.count_nonzero(label_gt[:,:,i]) > max_mask_value:
            max_mask_slice = i
            max_mask_value = np.count_nonzero(label_gt[:,:,i])

    fig, axes = plt.subplots(2, 5, figsize=(20, 8)) # Increased figure height

    for i, idx in enumerate(range(max_mask_slice - 2, max_mask_slice + 3)):
        if 0 <= idx < image.shape[2]:
            # Row 0: Image and label_gt
            axes[0, i].imshow(image[:, :, idx], cmap='gray')
            axes[0, i].imshow(label_gt[:, :, idx], alpha=0.2) # Reduced alpha for better visibility
            axes[0, i].set_title(f"Ground Truth segmentation Slice {idx}")
            axes[0, i].axis('off')

            # Row 1: Image and label_pred
            axes[1, i].imshow(image[:, :, idx], cmap='gray')
            axes[1, i].imshow(label_pred[:, :, idx], alpha=0.2) # Reduced alpha
            axes[1, i].set_title(f"Predicted Segmentation Slice {idx}")
            axes[1, i].axis('off')

        else:
            axes[0, i].axis('off')
            axes[1, i].axis('off')

    plt.tight_layout() # Adjust subplot params for a tight layout
    plt.show()




In [ ]:
import pandas as pd

dice_555 = pd.DataFrame({'patient_id': patient_id_555, 'dice_555': test_scores_dice_555})
dice_556 = pd.DataFrame({'patient_id': patient_id_556, 'dice_556': test_scores_dice_556})

merged_df = pd.merge(dice_555, dice_556, on='patient_id', how='outer')
merged_df.head(8)


In [ ]:
dataset = "Dataset557_OPCONTRASTNONCONTRAST"

imagesTs_dir = os.path.join('/content/runtime_files/data_dir/nnUNet_raw',dataset,'imagesTs')
labelsTs_gt_dir = os.path.join('/content/runtime_files/data_dir/nnUNet_raw',dataset,'labelsTs')

imagesTs_files = os.listdir(imagesTs_dir)
labelsTs_gt_files = os.listdir(labelsTs_gt_dir)


# results_dir =  ''
labelsTs_pred_dir = os.path.join(results_dir, dataset, 'test_predictions_3d_fullres')
labelsTs_pred_files = [filename for filename in os.listdir(labelsTs_pred_dir) if '.nii.gz' in filename]



# print(imagesTs_files)
# print(labelsTs_gt_files)
#print(labelsTs_pred_files)


with open(os.path.join(labelsTs_pred_dir, 'summary.json'), 'r') as f:
    test_results_summary_data = json.load(f)
    #print(data['foreground_mean'])
    print("OVERALL DICE SCORE: "+str(round(test_results_summary_data['foreground_mean']['Dice'],2)))
    print("OVERALL IOU SCORE: "+str(round(test_results_summary_data['foreground_mean']['IoU'],2)))

for imagesTs_file in imagesTs_files:
    print("\n\n")
    # CASE SPECIFIC SUMMARY & VISUALIZATION
    #print(imagesTs_file)
    label_name = imagesTs_file.replace('_0000','')
    #print(label_name)
    scan_id = "_"+str(label_name.split('_')[1].replace('.nii.gz',''))
    # print(scan_id)
    scan_id_int = int(label_name.split('_')[1].replace('.nii.gz',''))
    # print(scan_id_int)
    if scan_id_int <= 41:
        scan_id = "_"+str(label_name.split('_')[1].replace('.nii.gz',''))
    elif scan_id_int-41 <=9:
            scan_id = "_"+'00'+str(scan_id_int-41)
    elif scan_id_int-41 >9:
            scan_id = "_"+'0'+str(scan_id_int-41)



    patient_id = "IRB-0749:"+str(patient_ids_lookup[patient_ids_lookup['Patient_ID'] == scan_id]['patients'].iloc[0])
    print(patient_id)
    if scan_id_int <= 41:
        print("Contrast-enhanced")
    else:
        print("Non-contrast")
    for i in range(len(test_results_summary_data['metric_per_case'])):
        if scan_id == "_"+test_results_summary_data['metric_per_case'][i]['prediction_file'].split('_')[-1].replace(".nii.gz",""):
            print("DICE SCORE: "+str(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['Dice'],2)))
            print("IOU SCORE: "+str(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['IoU'],2)))
            # patient_id_556.append(patient_id)
            # test_scores_dice_556.append(round(test_results_summary_data['metric_per_case'][i]['metrics']['1']['Dice'],2))

    image = np.rot90(nib.load(os.path.join(imagesTs_dir, imagesTs_file)).get_fdata(), k=-1, axes=(0, 1))
    label_gt = np.rot90(nib.load(os.path.join(labelsTs_gt_dir, label_name)).get_fdata(), k=-1, axes=(0, 1))
    label_pred = np.rot90(nib.load(os.path.join(labelsTs_pred_dir, label_name)).get_fdata(), k=-1, axes=(0, 1))

    # print(image.shape)
    # print(label_gt.shape)
    # print(label_pred.shape)

    # find max mask slice and plot 3 slices above and below max mask slice

    max_mask_slice = 0
    max_mask_value = 0

    for i in range(image.shape[2]):
        if np.count_nonzero(label_gt[:,:,i]) > max_mask_value:
            max_mask_slice = i
            max_mask_value = np.count_nonzero(label_gt[:,:,i])

    fig, axes = plt.subplots(2, 5, figsize=(20, 8)) # Increased figure height

    for i, idx in enumerate(range(max_mask_slice - 2, max_mask_slice + 3)):
        if 0 <= idx < image.shape[2]:
            # Row 0: Image and label_gt
            axes[0, i].imshow(image[:, :, idx], cmap='gray')
            axes[0, i].imshow(label_gt[:, :, idx], alpha=0.3) # Reduced alpha for better visibility
            axes[0, i].set_title(f"Ground Truth segmentation Slice {idx}")
            axes[0, i].axis('off')

            # Row 1: Image and label_pred
            axes[1, i].imshow(image[:, :, idx], cmap='gray')
            axes[1, i].imshow(label_pred[:, :, idx], alpha=0.3) # Reduced alpha
            axes[1, i].set_title(f"Predicted Segmentation Slice {idx}")
            axes[1, i].axis('off')

        else:
            axes[0, i].axis('off')
            axes[1, i].axis('off')

    plt.tight_layout() # Adjust subplot params for a tight layout
    plt.show()


